<a href="https://colab.research.google.com/github/emguz/CSE-151A-Project/blob/main/CSE_151A_Project_hot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# CSV File Upload

In [ ]:
a = []
while(1):
    a.append('1')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# all_data_csv = glob.glob("data/*.csv")
from google.colab import files
# Choosing Kaggle key file from local computer
# To get a kaggle key, you have to log into your kaggle account and go to your account settings
# Scroll down to the API section and create a new token
# You should be able to download a file named 'kaggle' and this is the one you upload
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kekoapickett","key":"58c3cf46ccc730ac291b09d45f80dc39"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d mexwell/nba-shots

Dataset URL: https://www.kaggle.com/datasets/mexwell/nba-shots
License(s): other
100% 80.0M/80.4M [00:03<00:00, 33.1MB/s]
100% 80.4M/80.4M [00:03<00:00, 24.2MB/s]


In [6]:
!unzip \*.zip && rm *.zip

Archive:  nba-shots.zip
  inflating: NBA_2004_Shots.csv      
  inflating: NBA_2005_Shots.csv      
  inflating: NBA_2006_Shots.csv      
  inflating: NBA_2007_Shots.csv      
  inflating: NBA_2008_Shots.csv      
  inflating: NBA_2009_Shots.csv      
  inflating: NBA_2010_Shots.csv      
  inflating: NBA_2011_Shots.csv      
  inflating: NBA_2012_Shots.csv      
  inflating: NBA_2013_Shots.csv      
  inflating: NBA_2014_Shots.csv      
  inflating: NBA_2015_Shots.csv      
  inflating: NBA_2016_Shots.csv      
  inflating: NBA_2017_Shots.csv      
  inflating: NBA_2018_Shots.csv      
  inflating: NBA_2019_Shots.csv      
  inflating: NBA_2020_Shots.csv      
  inflating: NBA_2021_Shots.csv      
  inflating: NBA_2022_Shots.csv      
  inflating: NBA_2023_Shots.csv      
  inflating: NBA_2024_Shots.csv      


In [7]:
# Consolidating all csv files
import os
file_names = os.listdir()
all_data_csv = [file for file in file_names if file.endswith('.csv')]

In [8]:
dfs_combine = []

# for each file, add to dfs_combine
for file in all_data_csv:
    df = pd.read_csv(file)
    dfs_combine.append(df)

# join dfs together
df_all_data = pd.concat(dfs_combine, ignore_index=True)

# save combined dfs into new file
df_all_data.to_csv('all_years.csv', index=False)

In [9]:
# removes the individual year csv files from directory
for file in all_data_csv:
  os.remove(file)


# Exploration

In [10]:
data = pd.read_csv('all_years.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'all_years.csv'

In [ ]:
print('Data Columns: ')
print(data.columns)
print('Data Shape: ')
print(data.shape)

In [ ]:
data.head()

In [ ]:
for attribute in data.columns:
    print('-----' + attribute + '-----')
    print(data[attribute].unique())
    print('\n')

In [ ]:
# ['SEASON_1', 'SEASON_2', 'TEAM_ID', 'TEAM_NAME', 'PLAYER_ID',
#  'PLAYER_NAME', 'POSITION_GROUP', 'POSITION', 'GAME_DATE', 'GAME_ID',
#  'HOME_TEAM', 'AWAY_TEAM', 'EVENT_TYPE', 'SHOT_MADE', 'ACTION_TYPE',
#  'SHOT_TYPE', 'BASIC_ZONE', 'ZONE_NAME', 'ZONE_ABB', 'ZONE_RANGE',
#  'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER', 'MINS_LEFT', 'SECS_LEFT']

# data already clean -> separate into attribute categories

pred_attrib = ['EVENT_TYPE', 'SHOT_MADE']
numerical_attrib = ['SEASON_1', 'TEAM_ID', 'PLAYER_ID', 'GAME_ID', 'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER', 'MINS_LEFT', 'SECS_LEFT']
categorical_attrib = list(set(data.columns) - set(pred_attrib) - set(numerical_attrib))

In [ ]:
# heatmap w/ numerical attributes

# make figure larger
fig, ax = plt.subplots(figsize=(10,10))

heat_vars = numerical_attrib.copy()
heat_vars.append('SHOT_MADE')
sns.heatmap(data[heat_vars].corr(), annot=True,vmin=-1, vmax=1, center= 0, ax=ax)

In [ ]:
# pairplot with sample of data
sample = data.sample(int(len(data) * 0.0005))
sns.pairplot(sample, hue='SEASON_1', diag_kind='auto')

# Pre-Processing

In [ ]:
print(data.columns)

In [ ]:
# Dropping redundant attributes
Preprocessed_Data = data.drop('SEASON_2', axis=1)
Preprocessed_Data = Preprocessed_Data.drop('TEAM_NAME', axis=1)
Preprocessed_Data = Preprocessed_Data.drop('PLAYER_NAME', axis=1)
Preprocessed_Data = Preprocessed_Data.drop('POSITION', axis=1)
Preprocessed_Data = Preprocessed_Data.drop('GAME_DATE', axis=1)
Preprocessed_Data = Preprocessed_Data.drop('EVENT_TYPE', axis=1)
Preprocessed_Data = Preprocessed_Data.drop('ZONE_NAME', axis=1)

In [ ]:
# Encoding Categorical data

attributes_to_encode = ['POSITION_GROUP', 'HOME_TEAM', 'AWAY_TEAM', 'ACTION_TYPE', 'SHOT_TYPE', 'BASIC_ZONE', 'ZONE_ABB', 'ZONE_RANGE']

for attribute in attributes_to_encode:
  encoding = 0
  observations = Preprocessed_Data[attribute].unique()
  for observation in observations:
    if pd.isna(observation): # Checks for NAN
      Preprocessed_Data.loc[Preprocessed_Data[attribute].isna(), attribute] = -1
    else:
      Preprocessed_Data.loc[Preprocessed_Data[attribute] == observation, attribute] = encoding
      encoding += 1

In [ ]:
# Further Exploration

Preprocessed_Data.info()

In [ ]:
# heatmap w/ preprocessed attributes

# make figure larger
fig, ax = plt.subplots(figsize=(20,20))

attributes = list(Preprocessed_Data.columns)
heat_vars = attributes.copy()
sns.heatmap(Preprocessed_Data[heat_vars].corr(), annot=True,vmin=-1, vmax=1, center= 0, ax=ax)

In [ ]:
# Zone Range and shot type are highly correlated, dropping shot dist
Preprocessed_Data = Preprocessed_Data.drop(columns='SHOT_DISTANCE', axis=1)

# Also dropping Season_1 and Game_ID
Preprocessed_Data = Preprocessed_Data.drop(columns='GAME_ID', axis=1)
Preprocessed_Data = Preprocessed_Data.drop(columns='SEASON_1', axis=1)

In [ ]:
# Second heatmap to check for other highly correlated values

fig, ax = plt.subplots(figsize=(20,20))

attributes = list(Preprocessed_Data.columns)
heat_vars = attributes.copy()
sns.heatmap(Preprocessed_Data[heat_vars].corr(), annot=True,vmin=-1, vmax=1, center= 0, ax=ax)

# First Model

In [ ]:
# Splitting data (20/80 split)
y = Preprocessed_Data.SHOT_MADE
x = Preprocessed_Data.drop('SHOT_MADE', axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)

# Checking if our data has the right dimensions
print('xTrain shape:', x_train.shape)
print('yTrain shape:', y_train.shape)
print('xTest shape:', x_train.shape)
print('yTest shape:', y_test.shape)

In [ ]:
# Scaling data
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder

x_numerical_columns = ['TEAM_ID', 'PLAYER_ID', 'LOC_X', 'LOC_Y', 'QUARTER', 'MINS_LEFT', 'SECS_LEFT']
x_categorical_columns = ['POSITION_GROUP', 'HOME_TEAM', 'AWAY_TEAM', 'ACTION_TYPE', 'SHOT_TYPE', 'BASIC_ZONE', 'ZONE_ABB', 'ZONE_RANGE']

# Scaling numerical data
scaler = StandardScaler()
scaler.fit(Preprocessed_Data[x_numerical_columns])
# x_test_numerical = scaler.transform(x_test_numerical)



# Encoding categorical data
# make sparse_output=False to ensure a numpy array for an ouput, will be useful for concatenating data
# one_hot_encoder = OneHotEncoder(sparse_output=False)
preprocessed_hot = pd.get_dummies(Preprocessed_Data, columns= x_categorical_columns)
x_train, x_test, y_train, y_test = train_test_split(preprocessed_hot.drop('SHOT_MADE', axis=1), preprocessed_hot.SHOT_MADE, test_size=0.2, random_state=21)

x_train_numerical = scaler.transform(x_train[x_numerical_columns])
x_test_numerical = scaler.transform(x_test[x_numerical_columns])
x_train_numerical = pd.DataFrame(x_train_numerical, columns=x_numerical_columns, index=x_train.index)
x_test_numerical = pd.DataFrame(x_test_numerical, columns=x_numerical_columns, index=x_test.index)
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()
x_train_scaled[x_numerical_columns] = x_train_numerical
x_test_scaled[x_numerical_columns] = x_test_numerical



In [ ]:
print('Encoded and scaled data: ')
print(x_train_scaled.head())
print(' x_train shape: ', x_train_scaled.shape)
print(' x_test shape: ', x_test_scaled.shape)

In [ ]:
# Updating x_train and x_test arrays
x_train = x_train_scaled
x_test = x_test_scaled

# Checking if our data has the right dimensions
print('xTrain shape:', x_train.shape)
print('yTrain shape:', y_train.shape)
print('xTest shape:', x_test.shape)
print('yTest shape:', y_test.shape)

In [ ]:
# Training first model (Logistic Regression)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logisticModel = LogisticRegression(max_iter=1000)
logisticModel = logisticModel.fit(x_train, y_train)

# Evaluating Model
y_train_pred = logisticModel.predict(x_train)
y_test_pred = logisticModel.predict(x_test)

# Printing Reports

print(classification_report(y_test, y_test_pred))

In [ ]:
# TODO
# Fill in NAN values (making this a seperate ) [DONE]
# Second data exploration round, i.e. heat map [DONE]
# Get rid of attributes that correlate too much [DONE]
# Scale numerical attributes [DONE]
# One-hot-encode unordered(all of them) categorical values [DONE]
# Run first model (logistic regression) [DONE]
# Evaluate model, compare training vs test error [DONE]
# Answer 4-6 in write up

# Printing

In [ ]:
!apt-get update

In [ ]:
!pip install nbconvert
!apt-get install pandoc
!apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic -y

In [ ]:
!jupyter nbconvert --to pdf "/content/drive/MyDrive/CSE-151A-Project/CSE_151A_Project.ipynb"